In [1]:
import os

from pyecharts.charts import Bar
from pyecharts.charts import Graph
from pyecharts import options as opts

import pandas as pd

In [9]:
data_dir = "./weiboData_10000/1900232583/basic_info/"
os.getcwd()

'D:\\work\\GithubDesktopWork\\weibo-crawler'

In [10]:
user_data = pd.read_csv(data_dir + "user_id_name.csv")
user_data

,id,name
0,1900232583,俞静F
1,1971209421,Angela火星小姐姐
2,1632843173,认真的赵先森
3,1786937891,罐头辰
4,1735660651,杨帆-TJ
...,...,...
40131,5894594354,嘿豆豆豆子
40132,7471927686,人间帕罗西汀
40133,2951474854,易燃易爆易怼怼
40134,7288757608,扎西克鲁


In [11]:
user_data_id = user_data.set_index("id")
user_data_id

,name
id,
1900232583,俞静F
1971209421,Angela火星小姐姐
1632843173,认真的赵先森
1786937891,罐头辰
1735660651,杨帆-TJ
...,...
5894594354,嘿豆豆豆子
7471927686,人间帕罗西汀
2951474854,易燃易爆易怼怼


In [33]:
user_data_id.loc[1911140121]["name"]

KeyError: 1911140121

In [12]:
#微博关系的可视化
def draw_relations(data_dir):

    #读取id-name数据
    user_data_id = pd.read_csv(data_dir + "user_id_name.csv").set_index("id")

    visited_id_set = {}         #id-->index
    expanded_neighbor_set = {}      #源节点id->邻居节点ID-list
    categories = []
    node_list = []
    link_list = []

    #读取adj数据
    with open(data_dir + "user_id_adj.txt") as f:
        all_adj = f.readlines()

    #对于每一行数据
    for i in range(len(all_adj)):
        temp_adj = all_adj[i]
        temp_adj_id_list = temp_adj.strip().split("\t")
        temp_adj_name_list = [user_data_id.loc[int(temp_id)]["name"] for temp_id in temp_adj_id_list]

        #每一行设置对应类别
        categories.append(opts.GraphCategory(i))

        origin_id = int(temp_adj_id_list[0])
        #源节点邻居信息初始话
        expanded_neighbor_set[origin_id] = []

        print("origin node : {}".format(temp_adj_name_list[0]))

        #第一个节点
        if visited_id_set.__contains__(origin_id):
            print("first node {} visited".format(temp_adj_name_list[0]))
            node_list[visited_id_set[origin_id]] = opts.GraphNode(name=temp_adj_name_list[0], symbol_size=max(len(temp_adj_id_list) / 10, 2), category = i)
        else:
            visited_id_set[origin_id] = len(visited_id_set)
            node_list.append(opts.GraphNode(name=temp_adj_name_list[0], symbol_size=max(len(temp_adj_id_list) / 10, 2), category = i))


        for j in range(1, len(temp_adj_id_list)):
            temp_id_int = int(temp_adj_id_list[j])
            if not visited_id_set.__contains__(temp_id_int):
                visited_id_set[temp_id_int] = len(visited_id_set)
                node_list.append(opts.GraphNode(name=temp_adj_name_list[j], symbol_size=2, category = i))
            else:
                print("adj node {} visited".format(temp_adj_name_list[j]))

            #邻居信息的添加
            expanded_neighbor_set[origin_id].append(temp_id_int)

            #查看从邻居到源节点是否已经存在了，防止A->B, B->A记录两次
            if expanded_neighbor_set.__contains__(temp_id_int) and (origin_id in expanded_neighbor_set[temp_id_int]):
                print("edge {}->{} is duplicated".format(temp_adj_name_list[0], temp_adj_name_list[j]))
            else:
                #添加边
                link_list.append(opts.GraphLink(source=visited_id_set[origin_id], target=visited_id_set[temp_id_int]))

        print(temp_adj_name_list)
        print("*" * 100)
        # break

    # print(len(node_list))
    c = (
        Graph(init_opts=opts.InitOpts(width="1500px", height="700px", animation_opts=opts.AnimationOpts(animation=False)))
        .add(
            "",
            node_list,
            link_list,
            categories,
            repulsion=50,
            #可去掉，设置边的弧度
            linestyle_opts=opts.LineStyleOpts(curve=0.2),
            label_opts=opts.LabelOpts(is_show=False),
        )
        .set_global_opts(
            legend_opts=opts.LegendOpts(is_show=False),
            title_opts=opts.TitleOpts(title="weibo_data")
        )
        # .render_notebook()
        .render(data_dir + "./weibo_test.html")
    )

In [13]:
draw_relations(data_dir)

origin node : 俞静F
['俞静F', 'Angela火星小姐姐', '认真的赵先森', '罐头辰', '杨帆-TJ', '申哥_看大上海風情', '宏观对冲陈凯丰Kevin', '紫荆乡人李迪华', '建筑狂人']
****************************************************************************************************
origin node : Angela火星小姐姐
first node Angela火星小姐姐 visited
adj node 俞静F visited
edge Angela火星小姐姐->俞静F is duplicated
['Angela火星小姐姐', '海滨Carnival201701', 'Multi-Planet', '俞静F', '外太空总署']
****************************************************************************************************
origin node : 认真的赵先森
first node 认真的赵先森 visited
['认真的赵先森', '正大文西', '欣蕊Ray', 'Rabiiiit', '琳琳啃西瓜-', '三叶草ART', '我很有料', '唔啦啦啦呜啦啦啦', 'DK在北京', '关于银先生']
****************************************************************************************************
origin node : 罐头辰
first node 罐头辰 visited
['罐头辰', '世中刚刚好', '演员邵逸凡', '小枣子-努力画画', '王太若', '会魔法的卢卡斯', '八号电影', '诨名唤作巴布尔', '演员冯兵', '小蓝三号机', 'BioArt生物艺术', 'Kwok盖-', '上海白领张拍蒜', '阿司匹林和拉格朗日', '合金妖王刘成文', '万能的大熊', 'YHeroyR', '弃号125', '24小时健康加油站', '华迎美', '东北知识局', '我开心比

In [14]:
import numpy as np
data = np.load("./weiboData_10000/1900232583/basic_info/candidate_id_list.npy")
data[:10]

array([1797026712, 1876453085, 2558278633, 1978673852, 7553172730,
       1435230724, 1827750627, 1720874451, 5124333343, 1841273460],
      dtype=int64)